In [1]:
import os
import sklearn
import joblib
from sklearn.feature_extraction.text import CountVectorizer
import nltk
nltk.download('punkt')
nltk.download('stopwords')
import re
import nltk

import pandas as pd
import numpy as np

from bs4 import BeautifulSoup
from nltk.corpus import stopwords
nltk.download('wordnet')


from nltk import word_tokenize
from nltk.sentiment.util import mark_negation
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC
from nltk.tokenize.treebank import TreebankWordDetokenizer

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Hp\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Hp\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Hp\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
if __name__ == '__main__':
	# Get train data.
	train_data = pd.read_csv('SemEval2017-task4-dev.subtask-A.english.INPUT2.txt', sep="\t", header = None)

In [3]:
train_data.columns = ["id", "sentiment", "review", "time"]
train_data = train_data[["id", "sentiment", "review"]]
train_data.head()

,id,sentiment,review
0,619950566786113536,neutral,"Picturehouse's, Pink Floyd's, 'Roger Waters: T..."
1,619969366986235905,neutral,Order Go Set a Watchman in store or through ou...
2,619971047195045888,negative,If these runway renovations at the airport pre...
3,619974445185302528,neutral,If you could ask an onstage interview question...
4,619987808317407232,positive,A portion of book sales from our Harper Lee/Go...


In [4]:
train_data = train_data[train_data["sentiment"] != 'neutral']
train_data.head()

,id,sentiment,review
2,619971047195045888,negative,If these runway renovations at the airport pre...
4,619987808317407232,positive,A portion of book sales from our Harper Lee/Go...
5,619994586182619136,positive,"Excited to read ""Go Set a Watchman"" on Tuesday..."
6,619998879447162881,positive,Watching Miss USA tomorrow JUST to see @Travis...
8,620013074272137216,positive,Call for reservations for lunch or dinner tomo...


In [5]:
train_data = train_data.reset_index(drop=True)
train_data.head()

,id,sentiment,review
0,619971047195045888,negative,If these runway renovations at the airport pre...
1,619987808317407232,positive,A portion of book sales from our Harper Lee/Go...
2,619994586182619136,positive,"Excited to read ""Go Set a Watchman"" on Tuesday..."
3,619998879447162881,positive,Watching Miss USA tomorrow JUST to see @Travis...
4,620013074272137216,positive,Call for reservations for lunch or dinner tomo...


In [6]:
train_data["sentiment"] = (train_data["sentiment"] == 'positive')
train_data.head()

,id,sentiment,review
0,619971047195045888,False,If these runway renovations at the airport pre...
1,619987808317407232,True,A portion of book sales from our Harper Lee/Go...
2,619994586182619136,True,"Excited to read ""Go Set a Watchman"" on Tuesday..."
3,619998879447162881,True,Watching Miss USA tomorrow JUST to see @Travis...
4,620013074272137216,True,Call for reservations for lunch or dinner tomo...


In [7]:
import string
import re
import nltk

stopwords=nltk.corpus.stopwords.words('english')




In [8]:
def remove_punctuation(text):
    txt_nopunct= "".join([c for c in text if c not in string.punctuation])
    return txt_nopunct
train_data['no punct']=train_data['review'].apply(lambda x:remove_punctuation(x))

In [9]:
def tokenize(text):
    tokens=re.split('\W+',text)
    return tokens
train_data['token_text']=train_data['no punct'].apply(lambda x:tokenize(x))


In [10]:
def remove_stopwords(tokenized_text):
    clean_text=[word for word in tokenized_text if word not in stopwords]
    return clean_text;    
train_data['clean_text']=train_data['token_text'].apply(lambda x:remove_stopwords(x))


In [11]:
import nltk

wn=nltk.WordNetLemmatizer()
def lemmatization(text):
    text=[wn.lemmatize(word) for word in text]
    return text
train_data['lemmatized']=train_data['clean_text'].apply(lambda x:lemmatization(x))


In [12]:
def final(review_text):
    review_text = TreebankWordDetokenizer().detokenize(review_text)
    return review_text
train_data['final']=train_data['lemmatized'].apply(lambda x:final(x))


In [13]:
X = train_data["final"]
y = train_data["sentiment"]

y.head()

0    False
1     True
2     True
3     True
4     True
Name: sentiment, dtype: bool

In [14]:
X.head()

0    If runway renovation airport prevent seeing Ta...
1    A portion book sale Harper LeeGo Set Watchman ...
2    Excited read Go Set Watchman Tuesday But possi...
3    Watching Miss USA tomorrow JUST see TravisGarl...
4    Call reservation lunch dinner tomorrow yep Sun...
Name: final, dtype: object

In [15]:
from sklearn.model_selection import train_test_split

In [16]:
X_train_setup, X_test_setup, y_train, y_test = train_test_split(X, y, test_size=0.2)
X_train_setup = X_train_setup.reset_index(drop=True)
X_test_setup = X_test_setup.reset_index(drop=True)
y_train = y_train.reset_index(drop=True)
y_test = y_test.reset_index(drop=True)

In [17]:
X_train_setup.head()

0            I may started playing Kim Kardashian game
1    The hate eminating Muslims MuslimDragQueens hi...
2    If could win golf tournament 1st day Dustin Jo...
3    I dont care say Caitlyn Jenner may charged man...
4    superspurs34 Sgtshred1 Deadpool game coming PS...
Name: final, dtype: object

In [18]:
y_train.head()

0     True
1    False
2     True
3    False
4     True
Name: sentiment, dtype: bool

In [19]:
def process(X_data_setup, X_data):
    for index in range(0, len(X_data_setup)):
            review = X_data_setup[index]

            # Remove HTML tags.
            review_text = BeautifulSoup(review).get_text()
            # Convert all letters to lowercase.
            review_text = review_text.lower() 
            # Tokenize review.
            review_text = word_tokenize(review_text) 
            # Mark negations.
            review_text = mark_negation(review_text) 
            # Detokenize review.
            review_text = TreebankWordDetokenizer().detokenize(review_text)
            # Remove all non-letters.
            review_text = re.sub("[^a-zA-Z]"," ", review_text)
            X_data.append(review_text)

X_train = []
process(X_train_setup, X_train)
X_test=[]
process(X_test_setup, X_test)

In [20]:
X_test_setup

0       We beat Red Sox 31 Lets win series tomorrow af...
1             AntMan 30th AntMan Summer httptcoPfEUoXh1EI
2       AdamOsper think youre mad 6th possible Kane re...
3       I hope Oct 3 special Madison Square Garden fea...
4       A year ago today Kris Bryant named minor leagu...
                              ...                        
2053    Yakub may may hav deseved death definitely des...
2054    RedSox unbelievable bottom 4th Let go Red Sox ...
2055    Kim Kardashian may trashy sure hell smart busi...
2056    Kendrick giving free concert watt 22nd Whos go...
2057    We Im Happy place anyway congrats support Lady...
Name: final, Length: 2058, dtype: object

In [21]:
y_test

0        True
1        True
2       False
3        True
4        True
        ...  
2053    False
2054     True
2055     True
2056     True
2057     True
Name: sentiment, Length: 2058, dtype: bool

In [22]:
X_train

['i may started playing kim kardashian game',
 'the hate eminating muslims muslimdragqueens highlight incompatible disgusting religion  st world',
 'if could win golf tournament  st day dustin johnson would   major',
 'i dont care NEG say NEG caitlyn NEG jenner NEG may NEG charged NEG manslaughter NEG make NEG laugh NEG every NEG time NEG i NEG read NEG',
 'superspurs   sgtshred  deadpool game coming ps  xbox one november awesome',
 'vital testimony political character next prime minister george osborne httptco zikn uzao',
 'ya know john kasich actually seems like real human real record too bad may may mean anything',
 'jays starter david price suffers  st ever loss toronto      w jay     era    k    bb     ip    start',
 'blue jays  rd josh donaldson deserves al mvp sure and thats coming red sox fan that man fun watch',
 'have participated yet afriendlikeoyo contest oyorooms moto g  rd gen oyo vouchers shopping vouchers won',
 'sport paul dunne make history monday say jordan spieth fo

In [23]:
X_test

['we beat red sox    lets win series tomorrow afternoon mlb yankees',
 'antman   th antman summer httptcopfeuoxh ei',
 'adamosper think youre mad  th possible kane repeat last year feat next impossible',
 'i hope oct   special madison square garden feat brock lesnar broadcast network cause guy ringside seat',
 'a year ago today kris bryant named minor league player year today sits  th place rbis national league stud',
 'if i didnt training NEG tomorrow NEG id NEG sick NEG jason NEG aldean NEG concert NEG maybenexttime NEG',
 'jerk signed petition mercy yakub ensured doesnt go NEG no NEG httpstcoog zsdmwgw NEG',
 'ill watch  nd last eps naruto i watch new one',
 'now i understand george harrison wrote come sun sun almost month bangalore',
 'good luck tp irishman paul dunne final round open tomorrow httptcodiu  qzynf httptco wg  oixwb',
 'overalltonight reconfirmed love nicki i never went NEG back NEG side NEG bang NEG  th NEG grade NEG',
 'httptco iaiexuu u white sox wrong page matter c

In [24]:
train_data.head()

,id,sentiment,review,no punct,token_text,clean_text,lemmatized,final
0,619971047195045888,False,If these runway renovations at the airport pre...,If these runway renovations at the airport pre...,"[If, these, runway, renovations, at, the, airp...","[If, runway, renovations, airport, prevent, se...","[If, runway, renovation, airport, prevent, see...",If runway renovation airport prevent seeing Ta...
1,619987808317407232,True,A portion of book sales from our Harper Lee/Go...,A portion of book sales from our Harper LeeGo ...,"[A, portion, of, book, sales, from, our, Harpe...","[A, portion, book, sales, Harper, LeeGo, Set, ...","[A, portion, book, sale, Harper, LeeGo, Set, W...",A portion book sale Harper LeeGo Set Watchman ...
2,619994586182619136,True,"Excited to read ""Go Set a Watchman"" on Tuesday...",Excited to read Go Set a Watchman on Tuesday ...,"[Excited, to, read, Go, Set, a, Watchman, on, ...","[Excited, read, Go, Set, Watchman, Tuesday, Bu...","[Excited, read, Go, Set, Watchman, Tuesday, Bu...",Excited read Go Set Watchman Tuesday But possi...
3,619998879447162881,True,Watching Miss USA tomorrow JUST to see @Travis...,Watching Miss USA tomorrow JUST to see TravisG...,"[Watching, Miss, USA, tomorrow, JUST, to, see,...","[Watching, Miss, USA, tomorrow, JUST, see, Tra...","[Watching, Miss, USA, tomorrow, JUST, see, Tra...",Watching Miss USA tomorrow JUST see TravisGarl...
4,620013074272137216,True,Call for reservations for lunch or dinner tomo...,Call for reservations for lunch or dinner tomo...,"[Call, for, reservations, for, lunch, or, dinn...","[Call, reservations, lunch, dinner, tomorrow, ...","[Call, reservation, lunch, dinner, tomorrow, y...",Call reservation lunch dinner tomorrow yep Sun...


# RANDOM FOREST CLASSIFIER

In [25]:
# Creating Classifier using unigrams and bigrams.
# Altered bigram classifier.
from sklearn.ensemble import RandomForestClassifier

random_forest_bigram_classifier = Pipeline([('vectorizer', CountVectorizer(analyzer = "word",  ngram_range = (1, 2),  tokenizer = None, preprocessor = None, stop_words = None)),('classifier', RandomForestClassifier())])

random_forest_bigram_classifier.fit(X_train, y_train)

Pipeline(steps=[('vectorizer', CountVectorizer(ngram_range=(1, 2))),
                ('classifier', RandomForestClassifier())])

In [26]:
# Save model.
print("Saving Random Forest.")
filename = 'SemEvalModelRandomForest.sav'
#import pickle
joblib.dump(random_forest_bigram_classifier, filename)

Saving Random Forest.


['SemEvalModelRandomForest.sav']

In [27]:
# Loading classifier.
# Continue from here if model is already saved.
filename = 'SemEvalModelRandomForest.sav'
loaded_random_forest_bigram_classifier = joblib.load(filename)
print("Random Forest Classifier loaded.")

Random Forest Classifier loaded.


In [28]:
# For Random Forest.

y_train_pred = loaded_random_forest_bigram_classifier.predict(X_train)
# Create confusion matrix for X_train.
from sklearn.metrics import confusion_matrix

cm_train = confusion_matrix(y_train, y_train_pred)
# Find precision of the model.
total_train, correct_train = 0, 0
for i in range(0, len(cm_train)):
    for j in range(0, len(cm_train)):
        total_train += cm_train[i][j]
        if i == j:
            correct_train += cm_train[i][j]

precision_train = correct_train / total_train * 100.0
print("The train precision of the Random Forest Classifier model is " + str(precision_train))

The train precision of the Random Forest Classifier model is 99.98785228377065


In [29]:
print("Predicting test set data.")

Predicting test set data.


In [30]:
y_pred=random_forest_bigram_classifier.predict(X_test)
from sklearn.metrics import confusion_matrix

cm_train1 = confusion_matrix(y_test, y_pred)
# Find precision of the model.
total_train, correct_train = 0, 0
for i in range(0, len(cm_train1)):
    for j in range(0, len(cm_train1)):
        total_train += cm_train1[i][j]
        if i == j:
            correct_train += cm_train1[i][j]

precision_train = correct_train / total_train * 100.0
print("The test precision of the Random Forest Classifier model is " + str(precision_train))

The test precision of the Random Forest Classifier model is 82.94460641399417


# NAIVE BAYES

In [31]:
from sklearn.naive_bayes import MultinomialNB

naive_bayes_bigram_classifier = Pipeline([('vectorizer', CountVectorizer(analyzer = "word",  ngram_range = (1, 2),  tokenizer = None,  preprocessor = None,  stop_words = None)),('classifier', MultinomialNB())])

naive_bayes_bigram_classifier.fit(X_train, y_train)

# Save model.
print("Saving Naive Bayes model.")
filename = 'SemEvalModelNaiveBayes.sav'
#import pickle
joblib.dump(naive_bayes_bigram_classifier, filename)

# Loading classifier.
# Continue from here if model is already saved.
filename = 'SemEvalModelNaiveBayes.sav'
loaded_naive_bayes_bigram_classifier = joblib.load(filename)
print("Naive Bayes Classifier loaded.")

Saving Naive Bayes model.
Naive Bayes Classifier loaded.


In [32]:
# For Naive Bayes.

y_train_pred1 = loaded_naive_bayes_bigram_classifier.predict(X_train)
# Create confusion matrix for X_train.
from sklearn.metrics import confusion_matrix

cm_train2 = confusion_matrix(y_train, y_train_pred1)
# Find precision of the model.
total_train, correct_train = 0, 0
for i in range(0, len(cm_train2)):
    for j in range(0, len(cm_train2)):
        total_train += cm_train2[i][j]
        if i == j:
            correct_train += cm_train2[i][j]

precision_train = correct_train / total_train * 100.0
print("The train precision of the Naive Bayes model is " + str(precision_train))

The train precision of the Naive Bayes model is 98.01992225461613


In [33]:
print("Predicting test set data.")

Predicting test set data.


In [34]:
y_pred1=naive_bayes_bigram_classifier.predict(X_test)
from sklearn.metrics import confusion_matrix

cm_train3 = confusion_matrix(y_test, y_pred1)
# Find precision of the model.
total_train, correct_train = 0, 0
for i in range(0, len(cm_train3)):
    for j in range(0, len(cm_train3)):
        total_train += cm_train3[i][j]
        if i == j:
            correct_train += cm_train3[i][j]

precision_train = correct_train / total_train * 100.0
print("The test precision of the Naive Bayes Classifier model is " + str(precision_train))

The test precision of the Naive Bayes Classifier model is 84.59669582118562


In [35]:
y_pred1=naive_bayes_bigram_classifier.predict(X_test)
from sklearn.metrics import confusion_matrix

cm_train3 = confusion_matrix(y_test, y_pred1)
# Find precision of the model.
total_train, correct_train = 0, 0
for i in range(0, len(cm_train3)):
    for j in range(0, len(cm_train3)):
        total_train += cm_train3[i][j]
        if i == j:
            correct_train += cm_train3[i][j]

precision_train = correct_train / total_train * 100.0
print("The test precision of the Naive Bayes Classifier model is " + str(precision_train))

The test precision of the Naive Bayes Classifier model is 84.59669582118562


# SVM CLASSIFIER

In [36]:
bigram_classifier = Pipeline([('vectorizer', CountVectorizer(analyzer = "word",  ngram_range = (1, 2),  tokenizer = None,  preprocessor = None,  stop_words = None)),('classifier', LinearSVC())])

bigram_classifier.fit(X_train, y_train)

# Save model.
print("Saving model.")
filename = 'SemEvalModel.sav'
#import pickle
joblib.dump(bigram_classifier, filename)

# Loading classifier.
# Continue from here if model is already saved.
filename = 'SemEvalModel.sav'
loaded_bigram_classifier = joblib.load(filename)
print("SVC Classifier loaded.")

Saving model.
SVC Classifier loaded.


In [37]:
# SVM

y_train_pred2 = bigram_classifier.predict(X_train)
# Create confusion matrix for X_train.
from sklearn.metrics import confusion_matrix

cm_train4 = confusion_matrix(y_train, y_train_pred2)
# Find precision of the model.
total_train, correct_train = 0, 0
for i in range(0, len(cm_train4)):
    for j in range(0, len(cm_train4)):
        total_train += cm_train4[i][j]
        if i == j:
            correct_train += cm_train4[i][j]

precision_train = correct_train / total_train * 100.0
print("The train precision of the SVM model is " + str(precision_train))

The train precision of the SVM model is 100.0


In [38]:
print("Predicting test set data.")

Predicting test set data.


In [39]:
y_pred2=bigram_classifier.predict(X_test)
from sklearn.metrics import confusion_matrix

cm_train5 = confusion_matrix(y_test, y_pred2)
# Find precision of the model.
total_train, correct_train = 0, 0
for i in range(0, len(cm_train5)):
    for j in range(0, len(cm_train5)):
        total_train += cm_train5[i][j]
        if i == j:
            correct_train += cm_train5[i][j]

precision_train = correct_train / total_train * 100.0
print("The test precision of the SVM Classifier model is " + str(precision_train))

The test precision of the SVM Classifier model is 85.37414965986395


In [ ]:
	# GUI (Run from here to only run the user interface for a trained model)

	from tkinter import *

	root = Tk()
	root.title("Sentiment Predictor")

	reviewPrompt = Label(text = "Please enter your movie review below : ")
	reviewPrompt.pack(side = TOP)
	reviewRaw = Entry(root, width = 50)
	reviewRaw.pack(side = TOP, padx = 10, pady = 10)

	def getInput():
		textValue = reviewRaw.get()
		return textValue

	def findSentiment():

		# Get unclean review.
		unclean_review = getInput()
		print(unclean_review)

		# Predict sentiment.
		dummy_array, dummy_array_clean = [], []
		dummy_array.append(unclean_review)
		process(dummy_array, dummy_array_clean)
		print(dummy_array_clean[0])

		message = ""

		# SVC.
		message += "Support Vector : "
		prediction_svc = loaded_bigram_classifier.predict(dummy_array_clean)
		prediction_svc = prediction_svc[0]

		print(prediction_svc)
		if prediction_svc == 1:
			print("Positive.")
			message += "POSITIVE"
		else:
			print("Negative.")
			message += "NEGATIVE"

		# Random Forest.
		message += "\nRandom Forest : "
		prediction_svc = loaded_random_forest_bigram_classifier.predict(dummy_array_clean)
		prediction_svc = prediction_svc[0]

		print(prediction_svc)
		if prediction_svc == 1:
			print("Positive.")
			message += "POSITIVE"
		else:
			print("Negative.")
			message += "NEGATIVE"

		# Naive Bayes.
		message += "\nNaive Bayes : "
		prediction_svc = loaded_naive_bayes_bigram_classifier.predict(dummy_array_clean)
		prediction_svc = prediction_svc[0]

		print(prediction_svc)
		if prediction_svc == 1:
			print("Positive.")
			message += "POSITIVE"
		else:
			print("Negative.")
			message += "NEGATIVE"

		messagebox.showinfo("Predicted Polarity", message)


	#testMessage = Message(text = "Paad")
	#testMessage.config(bg = 'purple', font = ('courier', 12))

	senti = Button(root, text = "Find polarity.", command = findSentiment, bg = 'yellow')
	senti.pack()

	root.mainloop()
